In [1]:
import tensorflow as tf

In [2]:
tf.random.categorical(tf.math.log([[0.5, 0.5]]), 5)

<tf.Tensor: shape=(1, 5), dtype=int64, numpy=array([[1, 1, 1, 0, 1]], dtype=int64)>

In [1]:
import torch

c:\Users\rando\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
torch.sum(torch.mul(torch.randn((5, 10, 3)), torch.randn((1, 10, 1))), dim=1).shape

torch.Size([5, 3])